<a href="https://colab.research.google.com/github/Zr695/IMLO-OpenAssessment/blob/main/IMLO_OpenAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable

data_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    ])

training_data = datasets.Flowers102(
    root="data",
    split='train',
    download=True,
    transform=data_transform
)
validation_data = datasets.Flowers102(
    root="data",
    split='val',
    download=True,
    transform=data_transform
)
test_data = datasets.Flowers102(
    root="data",
    split='test',
    download=True,
    transform=data_transform
)


train_dataloader = DataLoader(training_data, batch_size=64)
validate_dataloader = DataLoader(validation_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.ReLU(),
          nn.Flatten(),
          nn.Dropout(p=0.2),
          nn.ReLU(),
          nn.Linear(16 * 250 * 250, 120),
          nn.ReLU(),
          nn.Linear(120, 84),
          nn.ReLU(),
          nn.Linear(84, 102),
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]

    def forward(self, x):
        logits = self.layers(x)
        return logits

model = NeuralNetwork()

In [3]:
learning_rate = 0.001
batch_size = 64
epochs = 5

In [4]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 2 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 4.688885  [   64/ 1020]
loss: 4.585338  [  128/ 1020]
loss: 4.609907  [  192/ 1020]
loss: 4.598594  [  256/ 1020]
loss: 4.616397  [  320/ 1020]
loss: 4.595493  [  384/ 1020]
loss: 4.608293  [  448/ 1020]
loss: 4.660731  [  512/ 1020]
loss: 4.687561  [  576/ 1020]
loss: 4.655262  [  640/ 1020]
loss: 4.622561  [  704/ 1020]
loss: 4.615527  [  768/ 1020]
loss: 4.660374  [  832/ 1020]
loss: 4.601003  [  896/ 1020]
loss: 4.649788  [  960/ 1020]
loss: 4.643619  [ 1020/ 1020]
Test Error: 
 Accuracy: 1.1%, Avg loss: 4.624421 

Epoch 2
-------------------------------
loss: 4.676630  [   64/ 1020]
loss: 4.595877  [  128/ 1020]
loss: 4.624774  [  192/ 1020]
loss: 4.597640  [  256/ 1020]
loss: 4.620416  [  320/ 1020]
loss: 4.600712  [  384/ 1020]
loss: 4.616163  [  448/ 1020]
loss: 4.656790  [  512/ 1020]
loss: 4.686615  [  576/ 1020]
loss: 4.644231  [  640/ 1020]
loss: 4.614683  [  704/ 1020]
loss: 4.605999  [  768/ 1020]
loss: 4.661230  [  832/ 1020]

KeyboardInterrupt: 